## Extracting TSS coordinates to calculate TSS-ATG distance distributions

This notebook explains how to extract  TSS coordinates from BED files and write the coordinates into a CSV file. The CSV file generated can be manipulated for subsequent analysis, for instance to calculate the distance distribution between TSSs and ATG start codons.

### Imports

In [ ]:
from Bio import SeqIO

### Extracting a list of TSS coordinates for each chromosome

The TSS map that we used (YeasTTS) contains all TSS coordinates in BED format. We downloaded the BED file for each chromosome and saved the file as "TSSs_chrA.bed", "TSSs_chrB.bed", "TSSs_chrC.bed" etc. From this BED file we then extracted a list of TSS coordinates for each chromosome using the code below.

In [ ]:
def get_TSSs(file):
    TSSs=[]
    
    for line in open(file,"r"):
        splits=line.split("\t")
        try:
            if splits[5]=='+':
                TSS_location=int(splits[1])
                TSSs.append([TSS_location, 1])
        except:
            pass
    for line in open(file,"r"):
        splits=line.split("\t")
        try:
            if splits[5]=='-':
                TSS_location=int(splits[2])
                TSSs.append([TSS_location, -1])
        except:
            pass
    return TSSs
        
for chr in ["chrA", "chrB", "chrC", "chrD", "chrE", "chrF"]:
    TSSs = get_TSSs("TSSs_"+chr+".bed")

### Define find current closest TSS

We next define a function that detects the closest TSS to the start codon of each gene. This function only searches for TSSs that are located within 2500bp of the ATG. Crucially, this function only assigns TSSs to start codons that have the same strandedness.   

In [ ]:
def current_closest_TSS(strand, loc_startcodon):
    minimum=100000000
    
    current_closest_TSS = None
    
    for TSS, TSS_strand in TSSs:
        if strand==1 and loc_startcodon-TSS <= minimum and loc_startcodon-TSS > 0 and TSS_strand==strand and loc_startcodon-TSS <= 2500:
            current_closest_TSS=TSS
            minimum=loc_startcodon-TSS

        if strand==-1 and TSS-loc_startcodon <= minimum and TSS-loc_startcodon > 0 and TSS_strand==strand and TSS-loc_startcodon <= 2500:
            current_closest_TSS=TSS
            minimum=TSS-loc_startcodon
            
    return current_closest_TSS, minimum

### Open CSV file

The next step involves creating a CSV file into which we will write any information of interest, e.g gene name, TSS locations and any distances of interest

In [ ]:
file_handle=open("Closest_TSS.csv", "w")
file_handle.write("GENE_NAME, CHROMOSOME, STRAND, START_CODON_LOC, TSS_LOC, TSS-TLS_DISTANCE, SEQUENCE\n")

### Write information into CSV file

To calculate the distance between TSS-ATG, we must first extract the ATG coordinates from an online database. We obtained coordinates from Genbank accession (GCA_001761485.1). Crucially, for the code below to work, each genbank file must be saved with the following format: 'Yali_chrA.gbk', 'Yali_chrB.gbk' etc. 

In [ ]:
# Loop over each chromosome
for chr in ["chrA", "chrB", "chrC", "chrD", "chrE", "chrF"]:
    TSSs = get_TSSs("TSSs_"+chr+".bed")
    seq_features=next(SeqIO.parse("Yali_"+chr+".gbk", "genbank")).features
    chromosome_sequence=str(next(SeqIO.parse("Yali_"+chr+".gbk", "genbank")).seq)
    
    for feature in seq_features:
        if feature.type=="CDS":
            strand=feature.location.strand 
            
            # Find TSS upstream of CDS for reverse strand
            if strand==-1:
                TSS, dist = current_closest_TSS(strand, feature.location.end)
                if TSS is not None:
                    file_handle.write(feature.qualifiers["locus_tag"][0]+","+chr+",-,"+str(feature.location.end)+","+str(TSS)+","+str(dist)+","+chromosome_sequence[TSS:TSS+2500]+"\n")
                
            # Find TSS upstream of CDS for forward strand
            else:
                TSS, dist = current_closest_TSS(strand, feature.location.start)
                if TSS is not None:
                    file_handle.write(feature.qualifiers["locus_tag"][0]+","+chr+",+,"+str(feature.location.start)+","+str(TSS)+","+str(dist)+","+chromosome_sequence[TSS-2500:TSS]+"\n")
        

To analyse a set of genes (e.g top 500 expressed genes) create a list of genes of interest. Next specify in an IF statement to only retain information if gene_name is in genes_of interest

In [ ]:
e.g     for feature in seq_features:
        if feature.type=="CDS":
            gene_name=feature.qualifiers["locus_tag"][0]
            if gene_name in genes_of_interest2:
                strand=feature.location.strand 

### Plot histogram of TSS-ATG distance 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.style.use('seaborn-dark-palette')

df = pd.read_csv("Closest_TSS.csv")
histogram = df[df.columns.values[5]]
plt.hist(histogram,bins=50)
plt.xlabel("TSS-ATG distance (bp)",color="black", fontsize=14 )
plt.ylabel("Frequency",color="black", fontsize=14)
plt.xticks(fontsize=13, color='black')
plt.yticks(fontsize=13, color='black')
plt.show()